In [1]:
import onnxruntime
import numpy as np
from transformers import BertTokenizer

onnx_file_path = "distilbert_model.onnx" # .onnx in the same directory
ort_session = onnxruntime.InferenceSession(onnx_file_path) 

tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')

def preprocess_text(text, max_sequence_length=512):
    tokens = tokenizer.encode(text, max_length=max_sequence_length, truncation=True)
    padding_length = max_sequence_length - len(tokens)
    tokens += [0] * padding_length  # Token d'ID de padding pour BERT
    return np.array(tokens).reshape(1, -1)

def predict_sentiment(text):
    input_ids = preprocess_text(text)

    ort_inputs = {'input_ids': input_ids}
    ort_outs = ort_session.run(None, ort_inputs)

    logits = ort_outs[0]
    predicted_class = np.argmax(logits, axis=1).item()

    return predicted_class

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [2]:
test_text = "This product exceeded my expectations! \
            It's incredibly effective, and I'm impressed with its quality. \
            I highly recommend it to anyone in need of a reliable solution."
predicted_sentiment = predict_sentiment(test_text)

if predicted_sentiment == 1:
    print("Sentiment positif")
else:
    print("Sentiment négatif")

Sentiment positif


In [3]:
test_text = "I  would not recommend this product. \
            The quality is poor, and it didn't work as expected. \
            I'm very disappointed with the purchase, and I regret buying it."
predicted_sentiment = predict_sentiment(test_text)

if predicted_sentiment == 1:
    print("Sentiment positif")
else:
    print("Sentiment négatif")

Sentiment négatif
